In [1]:
import sys
from glob import glob

import zarr
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#dataframe = pd.read_parquet(data_pdhrrr_dir+'grid_data_HRRR_d01_2020071200-0000.par', engine='pyarrow')

In [5]:
#dataframe.iloc[0:-1:1308]

In [6]:
#data = dataframe.values

In [7]:
#plt.plot(dataframe['torn_rptdist_2hr'].values[4:-1:1308])

In [8]:
from datetime import datetime, timedelta

In [13]:
base_v3_s = datetime(2018, 7, 15)
base_v3_e = datetime(2020, 12, 2)

base_v4_s = datetime(2020, 12, 3)
base_v4_e = datetime(2022, 7, 15)

base_ref = datetime(2010, 1, 1)

ref_v3 = (base_v3_e - base_ref).days

var_inds = [1, 46, 64, 76, 88, 106, 181]
filenames = sorted(glob(data_pdhrrr_dir+'*.par'))[:20]

N_var = len(var_inds)
N_grids = 1308
N_leads = 18
N_files = len(filenames)
N_samples = N_files*N_grids
N_cut = N_grids*N_leads

months = [2, 3, 4, 5, 6, 7, 8, 9, 10]

data_pos_v3 = np.empty((N_samples, N_leads, N_var))
data_neg_v3 = np.empty((N_samples, N_leads, N_var))
data_pos_v4 = np.empty((N_samples, N_leads, N_var))
data_neg_v4 = np.empty((N_samples, N_leads, N_var))

count_pos_v3 = 0
count_neg_v3 = 0
count_pos_v4 = 0
count_neg_v4 = 0

for i, filename in enumerate(filenames):
    
    dt_sring = filename[-19:-11]
    dt_ = datetime.strptime(dt_sring, '%Y%m%d')
    flag_v3 = (dt_ - base_ref).days < ref_v3
    
    if dt_.month in months is False:
        continue;
    else:
        dataframe = pd.read_parquet(filename, engine='pyarrow')
    
    if len(dataframe) < N_cut:
        continue;
    else:
        data = dataframe.values[:N_cut, var_inds]
    
    if flag_v3:
        for n in range(N_grids-1):
            data_slice = data[n:-1:1308, :]

            if np.max(data_slice[:, -1])>0 and np.max(data_slice[:, -1])<= 4000:
                data_pos_v3[count_pos_v3, ...] = data_slice
                count_pos_v3 += 1
            else:
                data_neg_v3[count_neg_v3, ...] = data_slice
                count_neg_v3 += 1
                
    else:
        for n in range(N_grids-1):
            data_slice = data[n:-1:1308, :]

            if np.max(data_slice[:, -1])>0 and np.max(data_slice[:, -1])<= 4000:
                data_pos_v4[count_pos_v4, ...] = data_slice
                count_pos_v4 += 1
            else:
                data_neg_v4[count_neg_v4, ...] = data_slice
                count_neg_v4 += 1   

In [10]:
data_pos_v3 = data_pos_v3[:count_pos_v3, ...]
data_neg_v3 = data_neg_v3[:count_neg_v3, ...]

data_pos_v4 = data_pos_v4[:count_pos_v4, ...]
data_neg_v4 = data_neg_v4[:count_neg_v4, ...]

zarr.save(save_dir_scratch+'HRRR_clean_pos_v3.zarr', data_pos_v3)
zarr.save(save_dir_scratch+'HRRR_clean_neg_v3.zarr', data_neg_v3)

zarr.save(save_dir_scratch+'HRRR_clean_pos_v4.zarr', data_pos_v4)
zarr.save(save_dir_scratch+'HRRR_clean_neg_v4.zarr', data_neg_v4)

In [14]:
np.max(data_pos_v3[:, :, -1])

4000.0

In [60]:

var_names = list(dataframe.columns)

for i, var_ in enumerate(var_names):
    print('{} - {}'.format(i, var_))

0 - fhr
1 - SBCAPE
2 - MLCAPE
3 - SBCINH
4 - MLCINH
5 - UP_HELI_MAX
6 - UP_HELI_MAX03
7 - W_UP_MAX
8 - W_DN_MAX
9 - SRH01
10 - SRH03
11 - SHR01
12 - SHR06
13 - CAPESHEAR
14 - T2
15 - TD2
16 - PSFC
17 - PREC_ACC_NC
18 - WSPD10MAX
19 - UP_HELI_MAX80
20 - UP_HELI_MAX120
21 - SBLCL
22 - STP
23 - U500
24 - V500
25 - T500
26 - TD500
27 - U700
28 - V700
29 - T700
30 - TD700
31 - U850
32 - V850
33 - T850
34 - TD850
35 - U925
36 - V925
37 - T925
38 - TD925
39 - LR75
40 - CREF
41 - GRPL_MAX
42 - HGT0C
43 - RVORT1
44 - MUCAPE
45 - UP_HELI_MAX-N3T1
46 - UP_HELI_MAX-N3T3
47 - UP_HELI_MAX-N3T5
48 - UP_HELI_MAX-N5T1
49 - UP_HELI_MAX-N5T3
50 - UP_HELI_MAX-N5T5
51 - UP_HELI_MAX03-N3T1
52 - UP_HELI_MAX03-N3T3
53 - UP_HELI_MAX03-N3T5
54 - UP_HELI_MAX03-N5T1
55 - UP_HELI_MAX03-N5T3
56 - UP_HELI_MAX03-N5T5
57 - RVORT1-N3T1
58 - RVORT1-N3T3
59 - RVORT1-N3T5
60 - RVORT1-N5T1
61 - RVORT1-N5T3
62 - RVORT1-N5T5
63 - W_UP_MAX-N3T1
64 - W_UP_MAX-N3T3
65 - W_UP_MAX-N3T5
66 - W_UP_MAX-N5T1
67 - W_UP_MAX-N5T3
68 - W